In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE

np.random.seed(42)

def generate_enhanced_data():
    num_resources = 100
    resource_data = {
        'resource_id': range(1, num_resources+1),
        'name': [f'Employee_{i}' for i in range(1, num_resources+1)],
        'department': np.random.choice(['IT', 'Engineering', 'Data Science', 'Operations'], num_resources),
        'skill_level': np.clip(np.random.normal(3, 1, num_resources).astype(int), 1, 5),
        'availability': np.clip(np.random.normal(30, 5, num_resources).astype(int), 20, 40),
        'current_workload': np.clip(np.random.normal(10, 5, num_resources).astype(int), 0, 20),
        'primary_skill': np.random.choice(['Python', 'Java', 'SQL', 'JavaScript', 'Cloud'], num_resources, p=[0.4, 0.2, 0.2, 0.15, 0.05]),
        'secondary_skill': np.random.choice(['Python', 'Java', 'SQL', 'JavaScript', 'Cloud', 'None'], num_resources, p=[0.3, 0.2, 0.2, 0.15, 0.05, 0.1]),
        'experience_years': np.clip(np.random.normal(5, 2, num_resources).astype(int), 1, 10)
    }

    num_tasks = 300
    task_data = {
        'task_id': range(1001, 1001+num_tasks),
        'project_name': [f'Project_{i%15}' for i in range(num_tasks)],
        'required_skill': np.random.choice(['Python', 'Java', 'SQL', 'JavaScript', 'Cloud'], num_tasks, p=[0.4, 0.2, 0.2, 0.15, 0.05]),
        'complexity': np.clip(np.random.normal(3, 1, num_tasks).astype(int), 1, 5),
        'estimated_hours': np.clip(np.random.normal(20, 10, num_tasks).astype(int), 5, 40),
        'priority': np.random.choice(['Critical', 'High', 'Medium', 'Low'], num_tasks, p=[0.1, 0.3, 0.4, 0.2]),
        'deadline_days': np.clip(np.random.normal(15, 7, num_tasks).astype(int), 1, 30)
    }

    resources = pd.DataFrame(resource_data)
    tasks = pd.DataFrame(task_data)

    tasks['complexity'] = np.where(tasks['priority'] == 'Critical',
                                 np.clip(tasks['complexity'] + 1, 1, 5),
                                 tasks['complexity'])

    return resources, tasks

def create_features(resources, tasks):
    combinations = []
    for _, task in tasks.iterrows():
        for _, resource in resources.iterrows():
            skill_match = 1 if (task['required_skill'] == resource['primary_skill'] or
                              task['required_skill'] == resource['secondary_skill']) else 0

            capacity_available = 1 if (resource['availability'] - resource['current_workload'] >=
                                     task['estimated_hours']) else 0

            skill_gap = max(0, task['complexity'] - resource['skill_level'])

            combinations.append({
                'task_id': task['task_id'],
                'resource_id': resource['resource_id'],
                'required_skill': task['required_skill'],
                'complexity': task['complexity'],
                'estimated_hours': task['estimated_hours'],
                'priority': task['priority'],
                'deadline_days': task['deadline_days'],
                'department': resource['department'],
                'skill_level': resource['skill_level'],
                'availability': resource['availability'],
                'current_workload': resource['current_workload'],
                'primary_skill': resource['primary_skill'],
                'secondary_skill': resource['secondary_skill'],
                'experience_years': resource['experience_years'],
                'skill_match': skill_match,
                'capacity_available': capacity_available,
                'skill_gap': skill_gap,
                'workload_ratio': resource['current_workload'] / max(1, resource['availability']),
                'priority_score': {'Critical':4, 'High':3, 'Medium':2, 'Low':1}[task['priority']]
            })

    return pd.DataFrame(combinations)

def create_target(df):
    """Create target with meaningful rules"""
    def calculate_target(row):
        base_score = 0
        base_score += 0.5 if row['skill_match'] == 1 else -0.3
        base_score += 0.4 if row['capacity_available'] == 1 else -0.4
        base_score -= 0.1 * row['skill_gap']
        base_score += 0.05 * min(3, row['experience_years'])
        base_score += 0.1 * (5 - row['priority_score'])

        base_score += np.random.normal(0, 0.1)

        return 1 if base_score >= 0.5 else 0

    return df.apply(calculate_target, axis=1)

def build_model():
    categorical_features = ['required_skill', 'priority', 'department', 'primary_skill', 'secondary_skill']
    numerical_features = ['complexity', 'estimated_hours', 'deadline_days', 'skill_level',
                         'availability', 'current_workload', 'experience_years', 'skill_gap',
                         'workload_ratio', 'priority_score']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])

    pipeline = make_pipeline(
        preprocessor,
        SMOTE(random_state=42),
        RandomForestClassifier(
            n_estimators=200,
            max_depth=10,
            min_samples_split=5,
            max_features='sqrt',
            class_weight='balanced_subsample',
            random_state=42,
            n_jobs=-1
        )
    )

    return pipeline

def train_model(X, y):
    model = build_model()

    print("Running cross-validation...")
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='f1', n_jobs=-1)
    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores):.2f}")

    print("\nTraining final model...")
    model.fit(X, y)

    return model

In [ ]:
# Generate data
print("Generating dataset...")
resources, tasks = generate_enhanced_data()

resources.to_csv('resources.csv', index=False)
tasks.to_csv('tasks.csv', index=False)
print("Saved raw datasets: resources.csv and tasks.csv")

Generating dataset...
Saved raw datasets: resources.csv and tasks.csv


In [ ]:
# Train model
print("Loading data...")
resources = pd.read_csv('resources.csv')
tasks = pd.read_csv('tasks.csv')

print("Creating features...")
allocation_data = create_features(resources, tasks)
allocation_data['good_allocation'] = create_target(allocation_data)

features = allocation_data.drop(columns=['task_id', 'resource_id', 'good_allocation'])
target = allocation_data['good_allocation']

print("Training model...")
model = train_model(features, target)

Loading data...
Creating features...
Training model...
Running cross-validation...
Cross-validation F1 scores: [0.79281184 0.80183372 0.80112157 0.79858363 0.79097194]
Mean CV F1 score: 0.80

Training final model...


In [ ]:
# Prediction
print("\nPredicting allocations...")
allocation_data['probability'] = model.predict_proba(features)[:, 1]

top_allocations = (allocation_data
                   .sort_values(['task_id', 'probability'], ascending=[True, False])
                   .drop_duplicates('task_id', keep='first'))

allocations = top_allocations[['task_id', 'resource_id', 'probability']]


final_allocations = allocations.merge(
    tasks[['task_id', 'project_name', 'required_skill', 'priority', 'estimated_hours', 'complexity']],
    on='task_id').merge(
    resources[['resource_id', 'name', 'department', 'primary_skill', 'availability', 'skill_level']],
    on='resource_id')

output_data = {
    'task_id': final_allocations['task_id'],
    'project_name': final_allocations['project_name'],
    'required_skill': final_allocations['required_skill'],
    'priority': final_allocations['priority'],
    'resource_id': final_allocations['resource_id'],
    'employee_name': final_allocations['name'],
    'department': final_allocations['department'],
    'employee_primary_skill': final_allocations['primary_skill'],
    'allocation_score': final_allocations['probability'],
    'estimated_hours': final_allocations['estimated_hours'],
    'employee_availability': final_allocations['availability'],
    'employee_skill_level': final_allocations['skill_level'],
    'task_complexity': final_allocations['complexity']
}

output_df = pd.DataFrame(output_data)

output_df.to_csv('optimal_allocations.csv', index=False)
print("Saved optimal allocations: optimal_allocations.csv")

print("\nFinal Model Performance:")
y_pred = model.predict(features)
print(classification_report(target, y_pred))

print("\nSample Optimal Allocations:")
print(output_df.sample(10).sort_values('allocation_score', ascending=False))


Predicting allocations...
Saved optimal allocations: optimal_allocations.csv

Final Model Performance:
              precision    recall  f1-score   support

           0       0.88      0.90      0.89     17502
           1       0.86      0.82      0.84     12498

    accuracy                           0.87     30000
   macro avg       0.87      0.86      0.86     30000
weighted avg       0.87      0.87      0.87     30000


Sample Optimal Allocations:
     task_id project_name required_skill priority  resource_id employee_name  \
208     1209   Project_13         Python      Low           28   Employee_28   
149     1150   Project_14         Python      Low           28   Employee_28   
290     1291    Project_5           Java      Low           52   Employee_52   
163     1164   Project_13           Java   Medium           52   Employee_52   
99      1100    Project_9           Java   Medium           26   Employee_26   
259     1260    Project_4           Java      Low           